In [1]:
import pandas as pd
import os
import ubiquant
import numpy as np
import pickle
import lightgbm as lgb
from ubq_utilities import STARTING_NUMERIC_FEAT_NAME
from ubq_nn_utils import create_mlp, PARAMS, tfp_correlation, create_ae_mlp
import re

path_model = '../input/ubq-ensemble-lgb-mlp'

env = ubiquant.make_env()   # initialize the environment

2022-04-15 06:34:15.588684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 06:34:15.594488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 06:34:15.595505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 06:34:15.597227: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
with open(os.path.join(path_model, 'ALL_FEATURE.pkl'), 'rb') as file:
    ALL_FEATURE = pickle.load(file)

ALL_FEATURE = ALL_FEATURE
SHAPE=len(ALL_FEATURE)

with open(os.path.join(path_model, 'PARAMS_AE.pkl'), 'rb') as file:
    PARAMS_AE = pickle.load(file)

with open(os.path.join(path_model, 'PARAMS_MLP.pkl'), 'rb') as file:
    PARAMS_MLP = pickle.load(file)

N_FOLD = 5

In [3]:
TIME_COL_LIST = [re.search('time_id_(.*)_mean', x).group(1) for x in ALL_FEATURE if 'time' in x]

In [4]:
def preprocess(data, time_col_list=TIME_COL_LIST):
    data['time_id'] = data['row_id'].str[0:4].astype(np.int64)
    for col in time_col_list:
        mapper_mean = data.groupby(['time_id'])[col].mean().to_dict()
        data[f'time_id_{col}_mean'] = data['time_id'].map(mapper_mean)
        data[f'time_id_{col}_mean'] = data[f'time_id_{col}_mean'].astype(np.float16)
    
    data['all_mean'] = data[STARTING_NUMERIC_FEAT_NAME].mean(axis=1)
    data['all_std'] = data[STARTING_NUMERIC_FEAT_NAME].std(axis=1)
    data['all_quantile_10'] = data[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.1, axis=1)
    data['all_quantile_50'] = data[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.5, axis=1)
    data['all_quantile_90'] = data[STARTING_NUMERIC_FEAT_NAME].quantile(q=0.9, axis=1)

    return data

# def get_ae_model_weights(fold, path_model=path_model):
    
#     model = create_ae_mlp(
#         **PARAMS_AE, steps=100,
#         metrics={'output': tfp_correlation},
#     )


#     model.load_weights(
#         os.path.join(path_model, f"model_ae_fold_{fold}.hdf5")
#     )
    
#     return model

def get_mlp_model_weights(fold, path_model=path_model):

    model = create_mlp(
        PARAMS_MLP['num_total_feature'], steps=100,
        metrics={'output': tfp_correlation},
    )


    model.load_weights(
        os.path.join(path_model, f"model_mlp_fold_{fold}.hdf5")
    )
    
    return model

def get_lgb_model(fold, prefix='', path_model=path_model):
    model = lgb.Booster(model_file=os.path.join(path_model, f'lgb{prefix}_fold_{fold}.txt'))
    return model


In [5]:
# model_ae_list = [
#     get_ae_model_weights(fold) for fold in range(N_FOLD)
# ]

model_mlp_list = [
    get_mlp_model_weights(fold) for fold in range(N_FOLD)
]

model_lgb_list = [
    get_lgb_model(fold, prefix='dart') for fold in range(N_FOLD)
]

In [6]:
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_values = preprocess(test_df)[ALL_FEATURE].to_numpy(dtype='float16')
    
    pred_lgb = [
        np.reshape(model.predict(test_values), (-1)) for model in model_lgb_list
    ]
    pred_mlp = [
        np.reshape(model.predict(test_values), (-1)) for model in model_mlp_list
    ]
#     pred_ae = [
#         np.reshape(model.predict(test_values)[-1], (-1)) for model in model_ae_list
#     ]
    pred = pred_lgb + pred_mlp# + pred_ae
    pred_blend = np.stack(pred).mean(axis=0)


    sample_prediction_df['target'] = pred_blend  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """
2022-04-15 06:34:24.043693: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
